In [1]:
from scipy.io.wavfile import read as read_wav
from IPython.display import Audio
import matplotlib.pyplot as plt
import tensorflow as tf
import sklearn as skl
import librosa as lr
import numpy as np
import logging
import random
import scipy
import os

In [2]:
np.random.seed(5)
logging.warnings.filterwarnings('ignore')

In [3]:
dataset_root = '..'

### Genre Dictionary:

In [4]:
genres = {
    'substrong1': 0,
    'substring2': 1
}

### Hyper Parameters:

In [5]:
hparams = {
    'samplerate': 22050,
    'seq_length': 256,
    'hop_length': 512,
    'fft_window': 2048,
    'num_classes': len(genres)
}

### Extract Features:

In [6]:
def extract_features(filepath, hparams):
    
    audio_data_array = lr.load(filepath, sr=hparams['samplerate'], mono=True)
    
    # normalized = lr.util.normalize(audio_data_array[0])
    
    features = [
        
        # MFCC Features:
        lr.feature.mfcc(
            y          = audio_data_array[0],
            sr         = hparams['samplerate'],
            hop_length = hparams['hop_length'],
            n_fft      = hparams['fft_window']
        ),

        # Chroma STFT:
        lr.feature.chroma_stft(
            y          = audio_data_array[0],
            sr         = hparams['samplerate'],
            hop_length = hparams['hop_length'],
            n_fft      = hparams['fft_window']
        ),
        
        # Spectral Centroid:
        lr.feature.spectral_centroid(
            y          = audio_data_array[0],
            sr         = hparams['samplerate'],
            hop_length = hparams['hop_length'],
            n_fft      = hparams['fft_window']
        ),
        
        # Spectral Contrast:
        lr.feature.spectral_contrast(
            y          = audio_data_array[0],
            sr         = hparams['samplerate'],
            hop_length = hparams['hop_length'],
            n_fft      = hparams['fft_window']
        )
    ]
    
    return np.vstack(features)

### Substrong-1 Classes Array:

In [7]:
substrong_1_classes = [
    'hiphop',
    'metal',
    'rock'
]

### Substrong-2 Classes Array:

In [8]:
substrong_2_classes = [
    'pop',
    'reggae'
]

### Allocating Arrays to Store Dataset:

In [9]:
train_data_x = np.zeros(shape=(5 * 350, 40, hparams['seq_length']))
val_data_x = np.zeros(shape=(5 * 100, 40, hparams['seq_length']))
test_data_x = np.zeros(shape=(5 * 50, 40, hparams['seq_length']))

train_data_y = np.zeros(shape=(5 * 350, 2))
val_data_y = np.zeros(shape=(5 * 100, 2))
test_data_y = np.zeros(shape=(5 * 50, 2))

### Train Dataset:

In [10]:
preprocess = True

if preprocess == True:

    train_data_counter = 0
    val_data_counter = 0
    test_data_counter = 0

    for genre in substrong_1_classes:

        print('Genre:', genre, 'Processed.')
        
        m = hparams['seq_length']

        filelist = os.listdir(dataset_root + '/dataset/' + genre)

        for file in filelist[0:70]:
            features = extract_features(dataset_root + '/dataset/' + genre + '/' + file, hparams)
            for i in range(5):
                train_data_x[train_data_counter, :] = features[:, (i * m): ((i+1) * m)]
                train_data_y[train_data_counter] = tf.keras.utils.to_categorical(0, num_classes=2)
                train_data_counter += 1
        
        for file in filelist[70:90]:
            features = extract_features(dataset_root + '/dataset/' + genre + '/' + file, hparams)
            for i in range(5):
                val_data_x[val_data_counter, :] = features[:, (i * m): ((i+1) * m)]
                val_data_y[val_data_counter] = tf.keras.utils.to_categorical(0, num_classes=2)
                val_data_counter += 1
                
        for file in filelist[90:100]:
            features = extract_features(dataset_root + '/dataset/' + genre + '/' + file, hparams)
            for i in range(5):
                test_data_x[test_data_counter, :] = features[:, (i * m): ((i+1) * m)]
                test_data_y[test_data_counter] = tf.keras.utils.to_categorical(0, num_classes=2)
                test_data_counter += 1
        

    for genre in substrong_2_classes:

        print('Genre:', genre, 'Processed.')
        
        m = hparams['seq_length']

        filelist = os.listdir(dataset_root + '/dataset/' + genre)

        for file in filelist[0:70]:
            features = extract_features(dataset_root + '/dataset/' + genre + '/' + file, hparams)
            for i in range(5):
                train_data_x[train_data_counter, :] = features[:, (i * m): ((i+1) * m)]
                train_data_y[train_data_counter] = tf.keras.utils.to_categorical(1, num_classes=2)
                train_data_counter += 1
        
        for file in filelist[70:90]:
            features = extract_features(dataset_root + '/dataset/' + genre + '/' + file, hparams)
            for i in range(5):
                val_data_x[val_data_counter, :] = features[:, (i * m): ((i+1) * m)]
                val_data_y[val_data_counter] = tf.keras.utils.to_categorical(1, num_classes=2)
                val_data_counter += 1
                
        for file in filelist[90:100]:
            features = extract_features(dataset_root + '/dataset/' + genre + '/' + file, hparams)
            for i in range(5):
                test_data_x[test_data_counter, :] = features[:, (i * m): ((i+1) * m)]
                test_data_y[test_data_counter] = tf.keras.utils.to_categorical(1, num_classes=2)
                test_data_counter += 1
            
    if 'prepared' not in os.listdir('.'):
        os.mkdir('prepared')
    np.save('./prepared/train_data_x.npy', train_data_x, allow_pickle=False)
    np.save('./prepared/train_data_y.npy', train_data_y, allow_pickle=False)
    np.save('./prepared/val_data_x.npy', val_data_x, allow_pickle=False)
    np.save('./prepared/val_data_y.npy', val_data_y, allow_pickle=False)
    np.save('./prepared/test_data_x.npy', test_data_x, allow_pickle=False)
    np.save('./prepared/test_data_y.npy', test_data_y, allow_pickle=False)
else:
    train_data_x = np.load('./prepared/train_data_x.npy', allow_pickle=False)
    train_data_y = np.load('./prepared/train_data_y.npy', allow_pickle=False)
    val_data_x = np.load('./prepared/val_data_x.npy', allow_pickle=False)
    val_data_y = np.load('./prepared/val_data_y.npy', allow_pickle=False)
    test_data_x = np.load('./prepared/test_data_x.npy', allow_pickle=False)
    test_data_y = np.load('./prepared/test_data_y.npy', allow_pickle=False)

Genre: hiphop Processed.
Genre: metal Processed.
Genre: rock Processed.
Genre: pop Processed.
Genre: reggae Processed.


In [11]:
train_data_x  = np.moveaxis(train_data_x, 1, 2)
val_data_x    = np.moveaxis(val_data_x,   1, 2)
test_data_x   = np.moveaxis(test_data_x,  1, 2)

### Model Creation and Training:

In [12]:
class GenreClassifierModel(tf.keras.Model):
    
    def __init__(self, l1, l2):
        
        super().__init__()
        
        self.L1 = tf.keras.layers.LSTM(l1, return_sequences=True)
        self.L2 = tf.keras.layers.LSTM(l2, return_sequences=False)
        self.L3 = tf.keras.layers.Dense(units=hparams['num_classes'], activation="softmax")
        
    def call(self, inputs):
        self.x = self.L1(inputs)
        self.x = self.L2(self.x)
        self.x = self.L3(self.x)
        return self.x

### Training Model:

In [13]:
def create_model(config):
    
    l1 = config['L1']
    l2 = config['L2']
    
    # Model Definition:
    model = GenreClassifierModel(l1, l2)
    
    # Model Compilation:
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [14]:
model = create_model({
    'L1': 64,
    'L2': 32
})

In [15]:
hist = model.fit(train_data_x, train_data_y, epochs=200)

Epoch 1/200
55/55 [==============================] - 1s 15ms/step - loss: 0.5698 - accuracy: 0.6886
Epoch 2/200
55/55 [==============================] - 1s 15ms/step - loss: 0.4803 - accuracy: 0.7611
Epoch 3/200
55/55 [==============================] - 1s 15ms/step - loss: 0.4623 - accuracy: 0.7777
Epoch 4/200
55/55 [==============================] - 1s 15ms/step - loss: 0.4502 - accuracy: 0.7851
Epoch 5/200
55/55 [==============================] - 1s 15ms/step - loss: 0.4227 - accuracy: 0.8000
Epoch 6/200
55/55 [==============================] - 1s 16ms/step - loss: 0.4366 - accuracy: 0.7977
Epoch 7/200
55/55 [==============================] - 1s 15ms/step - loss: 0.4027 - accuracy: 0.8189
Epoch 8/200
55/55 [==============================] - 1s 15ms/step - loss: 0.3979 - accuracy: 0.8206
Epoch 9/200
55/55 [==============================] - 1s 15ms/step - loss: 0.3938 - accuracy: 0.8160
Epoch 10/200
55/55 [==============================] - 1s 15ms/step - loss: 0.3889 - accuracy: 0.8217

55/55 [==============================] - 1s 16ms/step - loss: 0.2084 - accuracy: 0.9154
Epoch 83/200
55/55 [==============================] - 1s 16ms/step - loss: 0.1916 - accuracy: 0.9200
Epoch 84/200
55/55 [==============================] - 1s 16ms/step - loss: 0.2204 - accuracy: 0.9126
Epoch 85/200
55/55 [==============================] - 1s 16ms/step - loss: 0.1956 - accuracy: 0.9189
Epoch 86/200
55/55 [==============================] - 1s 16ms/step - loss: 0.1896 - accuracy: 0.9223
Epoch 87/200
55/55 [==============================] - 1s 16ms/step - loss: 0.1815 - accuracy: 0.9274
Epoch 88/200
55/55 [==============================] - 1s 16ms/step - loss: 0.1795 - accuracy: 0.9303
Epoch 89/200
55/55 [==============================] - 1s 16ms/step - loss: 0.2085 - accuracy: 0.9194
Epoch 90/200
55/55 [==============================] - 1s 16ms/step - loss: 0.1910 - accuracy: 0.9309
Epoch 91/200
55/55 [==============================] - 1s 16ms/step - loss: 0.2007 - accuracy: 0.9217
Epo

55/55 [==============================] - 1s 16ms/step - loss: 0.0872 - accuracy: 0.9680
Epoch 163/200
55/55 [==============================] - 1s 16ms/step - loss: 0.0858 - accuracy: 0.9714
Epoch 164/200
55/55 [==============================] - 1s 16ms/step - loss: 0.0838 - accuracy: 0.9686
Epoch 165/200
55/55 [==============================] - 1s 16ms/step - loss: 0.0670 - accuracy: 0.9743
Epoch 166/200
55/55 [==============================] - 1s 16ms/step - loss: 0.0707 - accuracy: 0.9754
Epoch 167/200
55/55 [==============================] - 1s 16ms/step - loss: 0.0950 - accuracy: 0.9680
Epoch 168/200
55/55 [==============================] - 1s 16ms/step - loss: 0.0702 - accuracy: 0.9777
Epoch 169/200
55/55 [==============================] - 1s 16ms/step - loss: 0.0821 - accuracy: 0.9714
Epoch 170/200
55/55 [==============================] - 1s 16ms/step - loss: 0.0850 - accuracy: 0.9714
Epoch 171/200
55/55 [==============================] - 1s 16ms/step - loss: 0.0769 - accuracy: 0

In [16]:
model.evaluate(val_data_x, val_data_y)

16/16 [==============================] - 0s 8ms/step - loss: 0.5072 - accuracy: 0.8760


[0.5071746706962585, 0.8759999871253967]

In [17]:
model.save('./models/model')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ./models/model/assets
